# Chance-constrained programming: portfolio optimization example

In [ ]:
using LinearAlgebra
using JuMP
using HiGHS, Gurobi
using Distributions
using Random

Adapted from Stephen Boyd and Lieven Vandenberghe, "Convex Optimization", Cambridge University Press, 2004, Section 4.7.6, p. 187.

We consider an investiment portfolio with $n$ assets and random returns, except the last one. We assume that the $n-1$ first returns follow a multivariate distribution with known mean and covariance matrix. The last asset is a risk-free investment product, with a fixed return.

The assets characteristics are detailed below.

In [ ]:
n = 4  # the last asset is a risk-free asset, with a null variance.
μ = [.12 ; .10 ; .07 ; .03]
Σ = [ 4e-2  6e-3 -4e-3  ;
      6e-3  1e-2  0.0  ;
      -4e-3  0.0 2.5e-3 ]

# We create une multivariate normal of mean μ and covariance matrix Σ
d = MvNormal(μ[1:n-1], Σ)

In [ ]:
# Estimate the probability to have a negative return and the resulting expected shortfall.
function expectedshortfall(p:: Vector, d:: Distribution, M:: Int = 1000000)
    
    loss = 0
    vloss = 0
    for i = 1:M
        ξ = [rand(d); μ[n]]
        ret = dot(p, ξ)
        if ret < 0
            loss += 1
            vloss += ret
        end
    end

    return loss/M, vloss/loss
    
end

## Portfolio with uniform repartition

We first consider the naive strategy where the same amount is invested in each asset.

In [ ]:
# Expected return with a uniform repartition.
p = ones(n)./n
er = sum(p[i]*μ[i] for i = 1:n)

We now compute the loss probability and the average loss when a loss occurs.

In [ ]:
expectedshortfall(p, d)

## Optimal decision without loss constraint

We now aim to maximize the expected return, without any consideration for the potential loss.

In [ ]:
m = Model(HiGHS.Optimizer)

@variable(m, p[1:n] >= -0.1)
@constraint(m, sum(p[i] for i = 1:n) <= 1)

@objective(m, Max, sum(p[i]*μ[i] for i = 1:n))

println(m)

In [ ]:
optimize!(m)

In [ ]:
value.(p)

Not surprisingly, we invest everything is the asset having the highest return rate, even borrowing from the lower return rate asset.

In [ ]:
objective_value(m)

The loss probability is however close to 30%, and the average lost amount is significantly more important if a loss occurs.

In [ ]:
expectedshortfall(value.(p), d)

## Optimal decision with loss constraint

We add the constraint that we want to limit the risk by accepting a loss with a maximum probability of 0.05.

In [ ]:
maxloss = 0

α = 0.95
z = 1/quantile(Normal(0,1), α)

We build the second-order cone constraint corresponding to the joint chance constraint.

In [ ]:
A = Σ^0.5

In [ ]:
A*A-Σ

Unfortunately, HiGHS does not support second-order cone constraint. We switch to Gurobi.

In [ ]:
set_optimizer(m, Gurobi.Optimizer)

In [ ]:
# || x || <= t, t >= 0
# https://jump.dev/JuMP.jl/stable/reference/constraints/#JuMP.SecondOrderCone
@constraint(m, [z*(-maxloss+sum(μ[i]*p[i] for i = 1:n)); (Σ^0.5)*p[1:n-1]] in SecondOrderCone())

println(m)

In [ ]:
optimize!(m)

In [ ]:
sol = value.(p)

In [ ]:
sol = value.(p)

In [ ]:
objective_value(m)

Without any surprise, the expected return is less than without the loss constraint, but is still higher than with the uniform repartition. We can also see that we use the risk-free asset as a borrowing tool. The risk to lose money is limited to 5%, as desired, and is less than any other strategy. Interestingly, the average loss is also the smallest one.

In [ ]:
expectedshortfall(value.(p), d)